In [1]:
import scan_csv
from LabelGenerator import *
from client import OIDClient
from FastSlidingWindow import *
from Util import *

ld = scan_csv.open_dicts()
train_image_codec = ld['train codec']


Loading dicts...
Total time in fractional seconds: 24.546875


In [2]:
print(ld.keys())

dict_keys(['class names', 'class encodings', 'train codec', 'train class', 'val codec', 'val class'])


In [3]:
car_images = ld['train class'][ld['class encodings']['Car']][100:110]
print(car_images[0])
print(len(car_images))

040c0777eb33f432
10


In [4]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.Session(config=config).close()

In [5]:
def calc_iou(Y_pred, Y):
    zeros = np.zeros([Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1], dtype=np.float32)
    y2 = tf.reshape(Y_pred[:, :, :, 2], [Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1])
    y4 = tf.reshape(Y_pred[:, :, :, 4], [Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1])
    pred_x_w = tf.where(y2 > 0.0, y2, zeros) 
    pred_x_h = tf.where(y4 > 0.0, y4, zeros) 
    
    x1_t = Y_pred[:, :, :, 1:2] - Y[:, :, :, 2:3] / 2.0
    x2_t = Y_pred[:, :, :, 1:2] + Y[:, :, :, 2:3] / 2.0
    
    y1_t = Y_pred[:, :, :, 3:4] - Y[:, :, :, 4:5] / 2.0
    y2_t = Y_pred[:, :, :, 3:4] + Y[:, :, :, 4:5] / 2.0
    
    x1_p = Y_pred[:, :, :, 1:2] - pred_x_w / 2.0
    x2_p = Y_pred[:, :, :, 1:2] + pred_x_w / 2.0
    
    y1_p = Y_pred[:, :, :, 3:4] - pred_x_h / 2.0
    y2_p = Y_pred[:, :, :, 3:4] + pred_x_h / 2.0
    
    cond1 = x2_t < x1_p
    cond2 = x2_p < x1_t
    cond3 = y2_t < y1_p
    cond4 = y2_p < y1_t
    cond_all = tf.logical_or(tf.logical_or(tf.logical_or(cond1, cond2), cond3), cond4)
    
    ious_np = np.zeros([Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2], 1], dtype=np.float32)
    
    '''
    far_x = np.min([x2_t, x2_p])
    near_x = np.max([x1_t, x1_p])
    far_y = np.min([y2_t, y2_p])
    near_y = np.max([y1_t, y1_p])
    '''
    
    far_x = tf.where(x2_t < x2_p, x2_t, x2_p)
    near_x = tf.where(x1_t > x1_p, x1_t, x1_p)
    far_y = tf.where(y2_t < y2_p, y2_t, y2_p)
    near_y = tf.where(y1_t > y1_p, y1_t, y1_p)
    
    inter_area = (far_x - near_x + 1.0) * (far_y - near_y + 1.0)
    true_box_area = (x2_t - x1_t + 1.0) * (y2_t - y1_t + 1.0)
    pred_box_area = (x2_p - x1_p + 1.0) * (y2_p - y1_p + 1.0)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    iou = tf.where(cond_all, ious_np, iou)
    return iou
    


In [6]:
from threading import Thread, Lock

class Gatherer(threading.Thread):
    def __init__(self, instances, ld, client, stride, size, overlap_ratio):
        self.instances = instances
        self.ld = ld
        self.client = client
        self.stride = stride
        self.size = size
        self.overlap_ratio = overlap_ratio
        threading.Thread.__init__(self)
        self.lock = Lock()
        self.items = []
        self.alive = True
    
    def run(self):
        index = 0
        while self.alive:
            instance = self.instances[index]
            index += 1
            if index >= len(self.instances):
                index = 0
            self.lock.acquire()
            if len(self.items) < 5:
                X, Y, image, cmap = label_image(instance, 
                                     self.ld['train codec'], 
                                     self.ld['class names'], 
                                     self.client, self.stride, self.size, self.overlap_ratio)
                self.items.append([X, Y, image, cmap])
            self.lock.release()
            
    def get_next(self):
        self.lock.acquire()
        items = [None, None, None, None]
        if len(self.items) > 0:
            items = self.items[0]
            self.items = self.items[1:]
        self.lock.release()
        return items

In [7]:
w_val = 0.01
W_M_1 = tf.Variable(np.random.uniform(-w_val, w_val, [4, 4, 3, 32]), dtype=tf.float32, name='WM1')
W_M_1_1 = tf.Variable(np.random.uniform(-w_val, w_val, [4, 4, 32, 64]), dtype=tf.float32, name='WM1')
W_M_2 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 64, 128]), dtype=tf.float32, name='WM2')
W_M_2_2 = tf.Variable(np.random.uniform(-w_val, w_val, [3, 3, 128, 64]), dtype=tf.float32, name='WM2')
W_M_3 = tf.Variable(np.random.uniform(-w_val, w_val, [7*7*64, 500]), dtype=tf.float32, name='WM3')
W_M_4_F = tf.Variable(np.random.uniform(-w_val, w_val, [500, 1*5]), dtype=tf.float32, name='WM4F')

In [8]:
client = OIDClient('192.168.1.31', 33333)
loss_value = tf.Variable(0.0, dtype=tf.float32)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
gatherer = Gatherer(car_images, ld, client, 30, 50, 0.1)
gatherer.start()

def mean_squared(inputs, targets):
  error = inputs - targets
  return tf.reduce_mean(tf.square(error))

index = 0
loss = 0.0
while True:
    items = gatherer.get_next()
    X, Y, image, cmap = items[0], items[1], items[2], items[3]
    
    
    #X, Y, image, cmap = label_image(instance, train_image_codec, ld['class names'], client, 30, 50)
    #X, Y, image, cmap = gather(car_images, ld, client, 30, 50)

    X = np.array(X, dtype=np.float32)
    if image is not None:
        if np.max([image.shape[0], image.shape[1]]) >= 2000:
            image = None

    if image is not None:
        # ONLY CAR
        Y = Y[:, :, 570:571, :]

        iou_sum = 0.0
        iou_mean = 0.0

        with tf.GradientTape() as tape:
            # ENCODE
            X_flat = tf.reshape(X, [X.shape[0]*X.shape[1], X.shape[2], X.shape[3], X.shape[4]])
            encode_conv = tf.nn.leaky_relu(tf.nn.conv2d(X_flat, W_M_1, [1, 2, 2, 1], 'SAME'))
            encode_conv = tf.nn.leaky_relu(tf.nn.conv2d(encode_conv, W_M_1_1, [1, 2, 2, 1], 'SAME'))
            share_conv = encode_conv

            share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2, [1, 1, 1, 1], 'SAME'))
            share_conv = tf.nn.leaky_relu(tf.nn.conv2d(share_conv, W_M_2_2, [1, 2, 2, 1], 'SAME'))
            share_flat = tf.reshape(share_conv, [share_conv.shape[0], W_M_3.shape[0]])

            flat_1 = tf.nn.leaky_relu(tf.matmul(share_flat, W_M_3))
            Y_pred = tf.matmul(flat_1, W_M_4_F)
            class_pred = tf.nn.sigmoid(Y_pred[:, 0])
            Y_pred = tf.stack([class_pred, Y_pred[:, 1], Y_pred[:, 2], Y_pred[:, 3], 
                               Y_pred[:, 4]], axis=1)
            Y_pred = tf.reshape(Y_pred, Y.shape)


            iou = calc_iou(Y_pred, Y)
            iou_1 = 1.0 - tf.reduce_mean(iou)

            ms = mean_squared(Y_pred, Y)
            loss_value = ms + iou_1

        loss += loss_value.numpy()
        params = [W_M_1, W_M_1_1, W_M_2, W_M_2_2, W_M_3, W_M_4_F]
        optimizer.apply_gradients(zip(tape.gradient(loss_value, params), params),
                            global_step=tf.train.get_or_create_global_step())

        img_orig = draw_from_label(image, Y, cmap, 50, 0.0, True)
        img_pred = draw_from_label(image, Y_pred.numpy(), cmap, 50, 0.9, True)

        #print(Y_pred.numpy()[5, 5])
        cv2.imshow('orig', img_orig)
        cv2.imshow('pred', img_pred)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            gatherer.alive = False
            break
    index += 1
    if index >= len(car_images):
        index = 0
        print(loss)
        loss = 0.0

Started client thread
0.9940811693668365
0.4548831582069397
0.9939253628253937
0.9863222539424896
0.9314422607421875
0.38616421818733215
0.3148595094680786
0.9652137160301208
0.6072561740875244
0.7276712357997894
0.7710805833339691
0.7762318849563599
0.3267209231853485
0.32151129841804504
0.622232973575592
0.6936630308628082
0.6406528949737549
0.5690055042505264
0.3722606897354126
0.6365240663290024
0.5311023443937302
0.9963875412940979
0.25592517852783203
0.23690806329250336
0.7663397490978241
0.21451298892498016
0.5857267677783966
0.3700622618198395
0.5050892233848572
0.6297265589237213
0.6000553965568542
0.6073879301548004
0.2493741810321808
0.24689097702503204
0.24917952716350555
0.8274091184139252
0.5741590708494186
0.23457828164100647
0.231856569647789
0.5497265756130219
0.448366641998291
0.5244719237089157
0.5517650693655014
0.7732511907815933
0.55660180747509
0.5437265783548355
0.5461185425519943
0.544670581817627
0.22475257515907288
0.5341722965240479
0.4411500245332718
0.5462